# 00a - Training YOLO11 Dog Pose Model

**FONDAMENTALE**: Questo notebook addestra il backbone YOLO per la detection di **CANI** con **24 keypoints anatomici**.

## Perché questo notebook è necessario

Il modello `yolo11n-pose.pt` standard è addestrato su **COCO Human Pose**:
- Rileva solo **persone** (classe 0)
- Estrae **17 keypoints umani** (naso, occhi, spalle, gomiti, polsi, anche, ginocchia, caviglie)

Per ResQPet serve un modello addestrato sul **Dog-Pose Dataset**:
- Rileva **cani**
- Estrae **24 keypoints anatomici del cane** (naso, occhi, orecchie, zampe, coda, etc.)

## Dataset

Il [Dog-Pose Dataset](https://docs.ultralytics.com/datasets/pose/dog-pose/) contiene:
- **6,773 immagini di training**
- **1,703 immagini di test**
- **24 keypoints** per cane con coordinate (x, y, visibility)

## Output

I modelli vengono salvati con **versioning automatico** in `ResQPet/weights/`:
- `yolo11n-dog-pose-v1.pt` - Prima versione
- `yolo11n-dog-pose-v2.pt` - Seconda versione (150 epochs, mAP@50=98.7%)
- etc.

## Struttura Path

```
ResQPet/
├── weights/                    # Modelli condivisi (NON in backend/)
│   ├── yolo11n-dog-pose-v2.pt  # Backbone attuale
│   ├── collar_detector.pt
│   └── ...
├── datasets/                   # Dataset per test
│   ├── dog-pose/
│   ├── stray-dogs-fyp/
│   └── ...
└── training/
    └── notebooks/              # Questo notebook
```

In [ ]:
# Installazione dipendenze
%pip install ultralytics torch torchvision onnx onnxruntime -q

In [ ]:
# Configurazione paths - RELATIVI per portabilità
import os
import sys
from pathlib import Path
from datetime import datetime
import shutil

import torch
from ultralytics import YOLO

# Determina la directory base del progetto (relativa al notebook)
# Struttura: ResQPet/training/notebooks/questo_notebook.ipynb
NOTEBOOK_DIR = Path.cwd()  # Directory corrente (dove si esegue il notebook)

# Se eseguito dalla cartella notebooks
if NOTEBOOK_DIR.name == "notebooks":
    PROJECT_DIR = NOTEBOOK_DIR.parent.parent  # ResQPet
elif NOTEBOOK_DIR.name == "training":
    PROJECT_DIR = NOTEBOOK_DIR.parent  # ResQPet
elif NOTEBOOK_DIR.name == "ResQPet":
    PROJECT_DIR = NOTEBOOK_DIR
else:
    # Fallback: cerca ResQPet nella struttura
    PROJECT_DIR = NOTEBOOK_DIR
    while PROJECT_DIR.name != "ResQPet" and PROJECT_DIR.parent != PROJECT_DIR:
        PROJECT_DIR = PROJECT_DIR.parent
    if PROJECT_DIR.name != "ResQPet":
        PROJECT_DIR = Path.cwd()  # Usa directory corrente come fallback

# Directory base che contiene ResQPet e i dataset
BASE_DIR = PROJECT_DIR.parent  # Contiene ResQPet, stray-dogs-detection, Stanford Dog, etc.

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"\nProject directory: {PROJECT_DIR}")
print(f"Base directory: {BASE_DIR}")

In [ ]:
# Configurazione paths - RELATIVI (CORRETTO)
# NOTA: weights è nella root del progetto, NON in backend/weights
WEIGHTS_DIR = PROJECT_DIR / "weights"  # Cartella modelli condivisa
RUNS_DIR = PROJECT_DIR / "training" / "runs" / "dog_pose"

WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)
RUNS_DIR.mkdir(parents=True, exist_ok=True)

# Determina versione modello (auto-incremento)
existing_models = list(WEIGHTS_DIR.glob("yolo11n-dog-pose*.pt"))
if existing_models:
    versions = []
    for m in existing_models:
        name = m.stem
        if "-v" in name:
            try:
                v = int(name.split("-v")[-1])
                versions.append(v)
            except:
                pass
    MODEL_VERSION = max(versions) + 1 if versions else 2
else:
    MODEL_VERSION = 1

MODEL_NAME = f"yolo11n-dog-pose-v{MODEL_VERSION}"

print(f"Project: {PROJECT_DIR}")
print(f"Weights: {WEIGHTS_DIR}")
print(f"Runs: {RUNS_DIR}")
print(f"\nModello output: {MODEL_NAME}.pt (versione {MODEL_VERSION})")

In [ ]:
# Configurazione training
# Rileva automaticamente il device migliore disponibile
if torch.cuda.is_available():
    DEVICE = 'cuda'
elif torch.backends.mps.is_available():
    DEVICE = 'mps'
else:
    DEVICE = 'cpu'

CONFIG = {
    'model': 'yolo11n-pose.pt',     # Modello base (human pose, verrà fine-tunato)
    'data': 'dog-pose.yaml',         # Dataset config (auto-download da Ultralytics)
    'epochs': 150,                   # Numero epoche
    'imgsz': 640,                    # Dimensione immagine
    'batch': 16,                     # Batch size (ridurre se OOM)
    'patience': 20,                  # Early stopping patience
    'device': DEVICE,                # Auto-detect: 'cuda', 'mps', o 'cpu'
    'workers': 4,                    # Data loader workers
    'optimizer': 'AdamW',            # Optimizer
    'lr0': 0.001,                    # Learning rate iniziale
    'lrf': 0.01,                     # Learning rate finale (fraction)
    'mosaic': 1.0,                   # Mosaic augmentation
    'mixup': 0.1,                    # Mixup augmentation
    'degrees': 10.0,                 # Rotation augmentation
    'translate': 0.1,                # Translation augmentation
    'scale': 0.5,                    # Scale augmentation
    'fliplr': 0.5,                   # Horizontal flip probability
}

print("Configurazione Training:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 1. Caricamento Modello Base

Partiamo da `yolo11n-pose.pt` (human pose) e lo fine-tuniamo sul Dog-Pose dataset.
Questo approccio di **transfer learning** è più efficiente che partire da zero.

In [ ]:
print("="*60)
print("CARICAMENTO MODELLO BASE")
print("="*60)

# Carica modello base (scarica automaticamente se non presente)
model = YOLO(CONFIG['model'])

print(f"\nModello caricato: {CONFIG['model']}")
print(f"Task: {model.task}")
print(f"\nInfo modello:")
print(model.info())

## 2. Verifica Dataset Dog-Pose

Il dataset verrà scaricato automaticamente da Ultralytics quando avviamo il training.
Verifichiamo prima la configurazione.

In [ ]:
# Mostra configurazione dataset dog-pose
print("="*60)
print("DOG-POSE DATASET")
print("="*60)

print("""
Il Dog-Pose Dataset contiene:

- Training: 6,773 immagini
- Test: 1,703 immagini
- Classi: 1 (Dog)
- Keypoints: 24

Keypoints anatomici del cane:
  0: nose
  1: left_eye
  2: right_eye  
  3: left_ear_base
  4: right_ear_base
  5: left_ear_tip
  6: right_ear_tip
  7: throat
  8: withers (garrese)
  9: left_front_elbow
 10: right_front_elbow
 11: left_front_knee
 12: right_front_knee
 13: left_front_paw
 14: right_front_paw
 15: left_back_elbow
 16: right_back_elbow
 17: left_back_knee
 18: right_back_knee
 19: left_back_paw
 20: right_back_paw
 21: tail_start
 22: tail_end
 23: chin

URL: https://docs.ultralytics.com/datasets/pose/dog-pose/
""")

## 3. Training

Avviamo il training. Il dataset verrà scaricato automaticamente.

**Tempo stimato:**
- Mac M1/M2: ~2-3 ore
- NVIDIA GPU: ~1-2 ore
- CPU: ~8-12 ore

In [ ]:
print("="*60)
print("AVVIO TRAINING YOLO DOG-POSE")
print("="*60)
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Device: {CONFIG['device']}")
print(f"Epochs: {CONFIG['epochs']}")
print("\nIl dataset Dog-Pose verrà scaricato automaticamente (~500MB)...")
print("="*60 + "\n")

# Avvia training
results = model.train(
    data=CONFIG['data'],
    epochs=CONFIG['epochs'],
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    patience=CONFIG['patience'],
    device=CONFIG['device'],
    workers=CONFIG['workers'],
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    fliplr=CONFIG['fliplr'],
    project=str(RUNS_DIR),
    name='yolo11n_dog_pose',
    exist_ok=True,
    verbose=True,
    plots=True,
    save=True
)

print("\n" + "="*60)
print("TRAINING COMPLETATO!")
print("="*60)

## 4. Valutazione

In [ ]:
# Trova il best model
best_model_path = RUNS_DIR / 'yolo11n_dog_pose' / 'weights' / 'best.pt'

if best_model_path.exists():
    print(f"Best model trovato: {best_model_path}")
    
    # Carica best model
    best_model = YOLO(str(best_model_path))
    
    # Valutazione su validation set
    print("\nValutazione su validation set...")
    metrics = best_model.val()
    
    print("\n" + "="*60)
    print("METRICHE VALUTAZIONE")
    print("="*60)
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP50-95: {metrics.box.map:.4f}")
    if hasattr(metrics, 'pose'):
        print(f"Pose mAP50: {metrics.pose.map50:.4f}")
        print(f"Pose mAP50-95: {metrics.pose.map:.4f}")
else:
    print(f"Best model non trovato: {best_model_path}")
    print("Esegui prima il training nella cella precedente.")

## 5. Test su Immagini

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Trova immagini di test dai dataset INTERNI al progetto
DATASETS_DIR = PROJECT_DIR / "datasets"

test_dirs = [
    DATASETS_DIR / "stray-dogs-fyp" / "images" / "testing",
    DATASETS_DIR / "dog-pose" / "test" / "images",
    DATASETS_DIR / "stanford-dogs" / "Images",
    DATASETS_DIR / "dog-with-leash" / "images",
]

test_images = []
for test_dir in test_dirs:
    if test_dir.exists():
        # Cerca jpg e png
        images = list(test_dir.glob('**/*.jpg'))[:3]
        images += list(test_dir.glob('**/*.png'))[:3]
        test_images.extend(images[:3])
        print(f"Trovate immagini in {test_dir.relative_to(PROJECT_DIR)}")

if not test_images:
    print("\nNessun dataset trovato. Struttura attesa:")
    print("  ResQPet/datasets/stray-dogs-fyp/images/testing/")
    print("  ResQPet/datasets/dog-pose/test/images/")
    print("  ResQPet/datasets/stanford-dogs/Images/")

print(f"\nTotale immagini test: {len(test_images)}")

In [ ]:
if best_model_path.exists() and test_images:
    best_model = YOLO(str(best_model_path))
    
    # Test su alcune immagini
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, img_path in enumerate(test_images[:6]):
        if i >= len(axes):
            break
            
        # Inference
        results = best_model(str(img_path), verbose=False)
        
        # Plot risultato
        result_img = results[0].plot()
        axes[i].imshow(result_img[..., ::-1])  # BGR to RGB
        
        # Info
        n_dogs = len(results[0].boxes) if results[0].boxes is not None else 0
        axes[i].set_title(f"{img_path.name}\n{n_dogs} dog(s) detected")
        axes[i].axis('off')
    
    # Nascondi assi vuoti
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.savefig(RUNS_DIR / 'test_results.png', dpi=150)
    plt.show()
else:
    print("Esegui prima il training o aggiungi immagini di test.")

In [ ]:
# Verifica keypoints estratti
if best_model_path.exists() and test_images:
    best_model = YOLO(str(best_model_path))
    
    # Test su prima immagine
    test_img = str(test_images[0])
    results = best_model(test_img, verbose=False)
    
    print("="*60)
    print("VERIFICA KEYPOINTS")
    print("="*60)
    print(f"\nImmagine: {test_img}")
    
    if results[0].keypoints is not None:
        kpts = results[0].keypoints.data[0].cpu().numpy()
        print(f"\nKeypoints shape: {kpts.shape}")
        print(f"Numero keypoints: {kpts.shape[0]}")
        print(f"\nPrimi 5 keypoints (x, y, confidence):")
        
        keypoint_names = [
            'nose', 'left_eye', 'right_eye', 'left_ear_base', 'right_ear_base',
            'left_ear_tip', 'right_ear_tip', 'throat', 'withers',
            'left_front_elbow', 'right_front_elbow', 'left_front_knee', 'right_front_knee',
            'left_front_paw', 'right_front_paw', 'left_back_elbow', 'right_back_elbow',
            'left_back_knee', 'right_back_knee', 'left_back_paw', 'right_back_paw',
            'tail_start', 'tail_end', 'chin'
        ]
        
        for i, kpt in enumerate(kpts[:5]):
            name = keypoint_names[i] if i < len(keypoint_names) else f'kpt_{i}'
            print(f"  {i:2d}. {name:20s}: x={kpt[0]:.1f}, y={kpt[1]:.1f}, conf={kpt[2]:.2f}")
        
        print(f"\n... e altri {kpts.shape[0] - 5} keypoints")
    else:
        print("\nNessun keypoint rilevato (nessun cane nell'immagine?)")

## 6. Export Modello

Esportiamo il modello in:
1. **PyTorch** (.pt) - Per uso nel backend Python
2. **ONNX** (.onnx) - Per deploy cross-platform

In [ ]:
print("="*60)
print("EXPORT MODELLO")
print("="*60)

if best_model_path.exists():
    best_model = YOLO(str(best_model_path))
    
    # 1. Copia PyTorch model nella cartella weights con versioning
    pt_output = WEIGHTS_DIR / f'{MODEL_NAME}.pt'
    shutil.copy(best_model_path, pt_output)
    print(f"\n[1/2] PyTorch model salvato: {pt_output}")
    print(f"      Dimensione: {pt_output.stat().st_size / 1024 / 1024:.2f} MB")
    
    # 2. Export ONNX
    print("\n[2/2] Esportazione ONNX...")
    onnx_output = best_model.export(
        format='onnx',
        imgsz=640,
        simplify=True,
        dynamic=False,
        opset=12
    )
    
    # Sposta ONNX nella cartella weights con stesso nome
    onnx_source = Path(onnx_output)
    onnx_dest = WEIGHTS_DIR / f'{MODEL_NAME}.onnx'
    if onnx_source.exists():
        shutil.move(str(onnx_source), str(onnx_dest))
        print(f"      ONNX model salvato: {onnx_dest}")
        print(f"      Dimensione: {onnx_dest.stat().st_size / 1024 / 1024:.2f} MB")
    
    print("\n" + "="*60)
    print("EXPORT COMPLETATO!")
    print("="*60)
    print(f"\nPer usare questo modello, aggiorna i config:")
    print(f"  BACKBONE_MODEL = WEIGHTS_DIR / '{MODEL_NAME}.pt'")
else:
    print("Best model non trovato. Esegui prima il training.")

## 7. Test Modello Esportato

In [ ]:
# Test modello PyTorch esportato
pt_model_path = WEIGHTS_DIR / f'{MODEL_NAME}.pt'

if pt_model_path.exists() and test_images:
    print(f"Test modello PyTorch esportato: {MODEL_NAME}.pt")
    
    exported_model = YOLO(str(pt_model_path))
    
    # Test
    results = exported_model(str(test_images[0]), verbose=False)
    
    print(f"\nRisultato:")
    print(f"  Boxes: {len(results[0].boxes) if results[0].boxes is not None else 0}")
    if results[0].keypoints is not None:
        print(f"  Keypoints shape: {results[0].keypoints.data.shape}")
        print(f"  Numero keypoints per detection: {results[0].keypoints.data.shape[1]}")
    
    print("\n Modello PyTorch funziona correttamente!")
else:
    print("Modello non trovato o nessuna immagine di test.")

In [ ]:
# Test modello ONNX
onnx_model_path = WEIGHTS_DIR / f'{MODEL_NAME}.onnx'

if onnx_model_path.exists() and test_images:
    print(f"Test modello ONNX esportato: {MODEL_NAME}.onnx")
    
    # YOLO può caricare direttamente ONNX
    onnx_model = YOLO(str(onnx_model_path))
    
    # Test
    results = onnx_model(str(test_images[0]), verbose=False)
    
    print(f"\nRisultato:")
    print(f"  Boxes: {len(results[0].boxes) if results[0].boxes is not None else 0}")
    if results[0].keypoints is not None:
        print(f"  Keypoints shape: {results[0].keypoints.data.shape}")
    
    print("\n Modello ONNX funziona correttamente!")
else:
    print("Modello ONNX non trovato o nessuna immagine di test.")

## 8. Aggiornamento Configurazione Backend

In [ ]:
print("="*60)
print("AGGIORNAMENTO CONFIGURAZIONE")
print("="*60)

print(f"""
Per usare il nuovo modello {MODEL_NAME}, aggiorna:

1. backend/app/config.py:
   
   MODELS = {{
       'backbone': WEIGHTS_DIR / '{MODEL_NAME}.pt',  # <-- AGGIORNATO
       'collar': WEIGHTS_DIR / 'collar_detector.pt',
       'skin': WEIGHTS_DIR / 'skin_classifier.pt',
       'pose': WEIGHTS_DIR / 'stray_pose_classifier.pt',
       'breed': WEIGHTS_DIR / 'breed_classifier.pt',
   }}

2. labeling_tool/config.py:
   
   BACKBONE_MODEL = WEIGHTS_DIR / '{MODEL_NAME}.pt'

3. Ri-esegui notebook 00_keypoints_extraction.ipynb
   per estrarre i 24 keypoints corretti dai dataset.

4. Ri-esegui notebook 03_pose_classifier.ipynb
   con input_dim = 72 (24 keypoints × 3).

5. Riavvia il backend:
   cd backend && python run.py
""")

## 9. Riepilogo

In [ ]:
print("\n" + "="*60)
print("RIEPILOGO TRAINING YOLO DOG-POSE")
print("="*60)

print(f"""
Dataset: Dog-Pose (Ultralytics)
  - Training: 6,773 immagini
  - Test: 1,703 immagini
  - Keypoints: 24 anatomici del cane

Modello Base: yolo11n-pose.pt (human pose)
  - Fine-tuned su dog-pose dataset
  - Transfer learning

Training:
  - Epochs: {CONFIG['epochs']}
  - Image Size: {CONFIG['imgsz']}
  - Batch Size: {CONFIG['batch']}
  - Device: {CONFIG['device']}

Output (versione {MODEL_VERSION}):
  - PyTorch: {WEIGHTS_DIR / f'{MODEL_NAME}.pt'}
  - ONNX: {WEIGHTS_DIR / f'{MODEL_NAME}.onnx'}

Keypoints del cane (24):
  - Testa: nose, eyes, ears, chin, throat
  - Corpo: withers (garrese)
  - Zampe anteriori: elbow, knee, paw (L/R)
  - Zampe posteriori: elbow, knee, paw (L/R)  
  - Coda: tail_start, tail_end

Prossimi passi:
  1. Aggiorna config.py per usare {MODEL_NAME}.pt
  2. Ri-esegui notebook 00 per estrarre keypoints
  3. Ri-esegui notebook 03 per addestrare pose classifier
  4. Riavvia backend
""")

# Verifica file creati
print("\nFile nella cartella weights:")
for f in sorted(WEIGHTS_DIR.glob("yolo11n-dog-pose*.pt")):
    size = f.stat().st_size / 1024 / 1024
    print(f"   {f.name}: {size:.2f} MB")